# Lab1: Ollama Lab

## 1.1 Structured Output

In [ ]:
#pip install -U ollama

In [ ]:
from ollama import chat
from pydantic import BaseModel

class Country(BaseModel):
  name: str
  capital: str
  languages: list[str]

response = chat(
  messages=[
    {
      'role': 'user',
      'content': 'Tell me about Thailand.',
    }
  ],
  model='llama3',
  format=Country.model_json_schema(),
)

country = Country.model_validate_json(response.message.content)
print(country)    

In [ ]:
from ollama import chat
from pydantic import BaseModel

class Pet(BaseModel):
  name: str
  animal: str
  age: int
  color: str | None
  favorite_toy: str | None

class PetList(BaseModel):
  pets: list[Pet]

response = chat(
  messages=[
    {
      'role': 'user',
      'content': '''
        I have two pets.
        A cat named Luna who is 5 years old and loves playing with yarn. She has grey fur.
        I also have a 2 year old black cat named Loki who loves tennis balls.
      ''',
    }
  ],
  model='llama3',
  format=PetList.model_json_schema(),
)

pets = PetList.model_validate_json(response.message.content)
print(pets)


## 1.2 Function Calling

In [11]:
def add_two_numbers(a: int, b: int) -> int:
  """
  Add two numbers

  Args:
    a: The first integer number
    b: The second integer number

  Returns:
    int: The sum of the two numbers
  """
  return a + b

In [ ]:
from ollama import chat
from ollama import ChatResponse


def add_two_numbers(a: int, b: int) -> int:
  """
  Add two numbers

  Args:
    a (int): The first number
    b (int): The second number

  Returns:
    int: The sum of the two numbers
  """
  return a + b


def subtract_two_numbers(a: int, b: int) -> int:
  """
  Subtract two numbers
  """
  return a - b


# Tools can still be manually defined and passed into chat
subtract_two_numbers_tool = {
  'type': 'function',
  'function': {
    'name': 'subtract_two_numbers',
    'description': 'Subtract two numbers',
    'parameters': {
      'type': 'object',
      'required': ['a', 'b'],
      'properties': {
        'a': {'type': 'integer', 'description': 'The first number'},
        'b': {'type': 'integer', 'description': 'The second number'},
      },
    },
  },
}

messages = [{'role': 'user', 'content': 'What is three minus one?'}]
print('Prompt:', messages[0]['content'])

available_functions = {
  'add_two_numbers': add_two_numbers,
  'subtract_two_numbers': subtract_two_numbers,
}

response: ChatResponse = chat(
  'llama3.2',
  messages=messages,
  tools=[add_two_numbers, subtract_two_numbers_tool],
)

if response.message.tool_calls:
  # There may be multiple tool calls in the response
  for tool in response.message.tool_calls:
    # Ensure the function is available, and then call it
    if function_to_call := available_functions.get(tool.function.name):
      print('Calling function:', tool.function.name)
      print('Arguments:', tool.function.arguments)
      output = function_to_call(**tool.function.arguments)
      print('Function output:', output)
    else:
      print('Function', tool.function.name, 'not found')

# Only needed to chat with the model using the tool call results
if response.message.tool_calls:
  # Add the function response to messages for the model to use
  messages.append(response.message)
  messages.append({'role': 'tool', 'content': str(output), 'name': tool.function.name})

  # Get final response from model with function outputs
  final_response = chat('llama3.2', messages=messages)
  print('Final response:', final_response.message.content)

else:
  print('No tool calls returned from model')

## 1.3 Chat with a Model

In [ ]:
from ollama import chat

messages = [
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
]

response = chat('llama3.2', messages=messages)
print(response['message']['content'])

In [ ]:
messages 

## 1.4 Ollama Embed - Generate embeddings with a model

In [32]:
from ollama import embed

response = embed(model='llama3.2', input='Hello, world!')
print(response['embeddings'])

[[0.004231493, 0.008082382, -0.00603481, -0.013145103, 0.013686095, 0.021414008, 0.018367574, -0.0009449855, -0.026470643, 0.0028262467, -0.018735897, 0.008421953, 0.014963668, 0.020890582, -0.008849035, -0.011149171, -0.016660279, -0.0018264423, 0.0041508237, -4.9876293e-05, -0.010656521, -0.009261315, 0.015698174, -0.014635768, 0.012885654, -0.0011558665, 0.0059832563, -0.015735487, 0.029185193, 0.0184418, -0.014618216, -0.009436516, 0.020217443, 0.0012864098, 0.012320371, -0.008525519, -0.0029650657, 0.011919606, -0.008559848, -0.009390561, 0.0064799665, 0.0027786375, -0.0064937286, -0.0013167799, -0.019921098, -0.0030325556, -0.012136703, 0.010600206, 0.0073048086, -0.02388942, 0.016451983, 0.002706357, -0.026397647, 0.022138678, -0.0072055543, -0.014059758, 0.01517934, -0.02993396, 0.008412484, 0.002258148, -0.001294634, 0.02595166, -0.019347914, -0.013281413, 0.036186665, -0.030811477, 0.0036368084, -0.013873948, -0.0034489704, -0.010714449, 0.0013053436, -0.0087991785, -0.002910

In [1]:
from ollama import list
from ollama import ListResponse

response: ListResponse = list()

for model in response.models:
  print('Name:', model.model)
  print('  Size (MB):', f'{(model.size.real / 1024 / 1024):.2f}')
  if model.details:
    print('  Format:', model.details.format)
    print('  Family:', model.details.family)
    print('  Parameter Size:', model.details.parameter_size)
    print('  Quantization Level:', model.details.quantization_level)
  print('\n')

Name: llama3.1:latest
  Size (MB): 4692.80
  Format: gguf
  Family: llama
  Parameter Size: 8.0B
  Quantization Level: Q4_K_M


Name: llama3.2:latest
  Size (MB): 1925.84
  Format: gguf
  Family: llama
  Parameter Size: 3.2B
  Quantization Level: Q4_K_M


Name: nomic-embed-text:latest
  Size (MB): 261.60
  Format: gguf
  Family: nomic-bert
  Parameter Size: 137M
  Quantization Level: F16


Name: llama3:latest
  Size (MB): 4445.29
  Format: gguf
  Family: llama
  Parameter Size: 8B
  Quantization Level: Q4_0


Name: toche7/my-openthaigpt:latest
  Size (MB): 3919.47
  Format: gguf
  Family: llama
  Parameter Size: 7B
  Quantization Level: Q4_0


